In [ ]:
from ultralytics import YOLO
import os
import cv2
from tqdm import tqdm
import json
from matplotlib import pyplot as plt

# Ensure Dataset Path Exists
dataset_path = "dataset/data.yaml"  # Update with your dataset path
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset YAML file not found: {dataset_path}")

# Load Pretrained YOLOv8 Model
model = YOLO('yolov8n.pt')  # Use 'yolov8n.pt', 'yolov8s.pt', etc., based on your hardware capacity

# Train the Model
results = model.train(
    data=dataset_path,       # Path to dataset YAML
    epochs=20,                # Number of epochs
    batch=16,                # Batch size
    imgsz=640,               # Image size
    workers=4,               # Number of data loading workers
    optimizer='AdamW',       # Optimizer
    lr0=0.001,               # Initial learning rate
    patience=10,             # Early stopping
    augment=True,            # Data augmentation
    val=True                 # Validate after each epoch
)

# Plot Training Loss
if hasattr(results, 'metrics'):
    metrics = results.metrics
    plt.figure(figsize=(10, 5))
    plt.plot(metrics.get('box_loss', []), label="Box Loss")
    plt.plot(metrics.get('obj_loss', []), label="Object Loss")
    plt.plot(metrics.get('cls_loss', []), label="Classification Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training Loss Over Epochs")
    plt.legend()
    plt.show()
else:
    print("Training metrics are not available for plotting.")

# Save the Best Model
best_model_path = 'chair_detection/yolov8_chair/weights/best.pt'
os.makedirs(os.path.dirname(best_model_path), exist_ok=True)
model.save(best_model_path)
print(f"Training complete. Best model saved at {best_model_path}")

# Load Trained Model
model = YOLO(best_model_path)

# Function to Detect and Count Objects
def detect_and_count(image, model, class_name="chair"):
    results = model(image)  # Run detection
    detections = results[0].boxes  # Extract bounding boxes
    chair_count = 0

    # Process each detection
    for detection in detections:
        class_id = int(detection.cls)  # Class ID
        if model.names[class_id] == class_name:  # Match the target class
            chair_count += 1
            # Draw bounding box and label
            x1, y1, x2, y2 = map(int, detection.xyxy[0])  # Bounding box coordinates
            conf = detection.conf  # Confidence score
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(
                image,
                f"{class_name} {conf:.2f}",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                2,
            )
    return image, chair_count

# Process Images and Save Results
def process_images(input_dir, output_dir, model, class_name="chair"):
    results_summary = []
    supported_formats = (".jpg", ".png", ".jpeg")
    os.makedirs(output_dir, exist_ok=True)

    for img_file in tqdm(os.listdir(input_dir), desc="Processing images"):
        if not img_file.endswith(supported_formats):
            continue

        img_path = os.path.join(input_dir, img_file)
        try:
            image = cv2.imread(img_path)
            if image is None:
                raise ValueError("Invalid image format or file.")

            annotated_image, count = detect_and_count(image, model, class_name)
            output_path = os.path.join(output_dir, img_file)
            cv2.imwrite(output_path, annotated_image)
            results_summary.append({"image": img_file, "chair_count": count})

        except Exception as e:
            print(f"Error processing {img_file}: {e}")

    summary_path = os.path.join(output_dir, "results_summary.json")
    with open(summary_path, "w") as json_file:
        json.dump(results_summary, json_file, indent=4)

    print(f"Processing complete. Results saved to {summary_path}")

# Live Detection Display
def live_detection(input_dir, model, class_name="chair"):
    supported_formats = (".jpg", ".png", ".jpeg")
    window_name = "Live Detection Results"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, 800, 600)

    for img_file in tqdm(os.listdir(input_dir), desc="Live Detection"):
        if not img_file.endswith(supported_formats):
            continue

        img_path = os.path.join(input_dir, img_file)
        try:
            image = cv2.imread(img_path)
            if image is None:
                raise ValueError("Invalid image format or file.")

            annotated_image, count = detect_and_count(image, model, class_name)
            cv2.imshow(window_name, annotated_image)
            if cv2.waitKey(500) & 0xFF == ord('q'):  # Press 'q' to exit
                break

        except Exception as e:
            print(f"Error processing {img_file}: {e}")

    cv2.destroyAllWindows()

# Save Detections as Video
def save_detection_video(input_dir, output_video_path, model, class_name="chair"):
    supported_formats = (".jpg", ".png", ".jpeg")
    frame_rate = 10  # Adjust frame rate as needed

    # Validate input images
    valid_images = [
        os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(supported_formats)
    ]
    if not valid_images:
        raise ValueError("No valid image files found in the directory.")

    # Determine frame size
    first_frame = cv2.imread(valid_images[0])
    height, width, _ = first_frame.shape

    # Define video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    for img_file in tqdm(valid_images, desc="Creating Video"):
        try:
            image = cv2.imread(img_file)
            annotated_image, count = detect_and_count(image, model, class_name)
            video_writer.write(annotated_image)

        except Exception as e:
            print(f"Error processing {img_file}: {e}")

    video_writer.release()
    print(f"Video saved at {output_video_path}")

# Input and Output Directories
INPUT_IMAGES_DIR = "dataset/images/test"  # Replace with your image directory
OUTPUT_IMAGES_DIR = "detected"  # Replace with output directory
os.makedirs(OUTPUT_IMAGES_DIR, exist_ok=True)

# Process and Save Annotated Images
process_images(INPUT_IMAGES_DIR, OUTPUT_IMAGES_DIR, model)

# Live Detection Display
live_detection(INPUT_IMAGES_DIR, model)

# Save Detection Results as Video
save_detection_video(INPUT_IMAGES_DIR, "detections_output.mp4", model)


100%|██████████| 6.25M/6.25M [00:03<00:00, 1.78MB/s]


New https://pypi.org/project/ultralytics/8.3.48 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.47  Python-3.12.6 torch-2.5.1+cpu CPU (Intel Core(TM) i7-4510U 2.00GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset/data.yaml, epochs=1, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False,

train: Scanning C:\Users\davoo\OneDrive\Desktop\Computer Vision\Project\Computer-Vision-24\Project\dataset\labels\train... 280 images, 7 backgrounds, 0 corrupt: 100%|██████████| 280/280 [00:04<00:00, 65.20it/s]


train: New cache created: C:\Users\davoo\OneDrive\Desktop\Computer Vision\Project\Computer-Vision-24\Project\dataset\labels\train.cache


val: Scanning C:\Users\davoo\OneDrive\Desktop\Computer Vision\Project\Computer-Vision-24\Project\dataset\labels\valid... 80 images, 1 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:01<00:00, 57.93it/s]

...


0: 480x640 (no detections), 331.6ms
Speed: 0.0ms preprocess, 331.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)


Creating Video:  98%|█████████▊| 39/40 [00:38<00:00,  1.01it/s]


0: 480x640 (no detections), 401.0ms
Speed: 10.9ms preprocess, 401.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)


Creating Video: 100%|██████████| 40/40 [00:39<00:00,  1.00it/s]

Video saved at detections_output.mp4
